In [23]:
import mysql
import mysql.connector
from selenium import *
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import sys, time
import pandas as pd

# ====================== #

# # host 정보 확인
# host = int(input('which is your host? 1. local / 2. server  >> '))
# if host == 1:
#     host = '192.168.35.159'
# elif host == 2:
#     host = input('input server address: ')
# else:
#     sys.exit()
    
mydb = mysql.connector.connect(host = '192.168.35.159', port = '3306', user='', password = '')
mydb

mycursor = mydb.cursor(buffered = True)

mycursor.execute('USE lotto_db')

# DB에 저장된 모든 회차 정보 조회
mycursor.execute('SELECT ID FROM lotto_nums ORDER BY ID DESC')
all_num_in_sql = mycursor.fetchall()
all_num_in_sql = [all_num_in_sql[i][0] for i in range(0,len(all_num_in_sql))]
# print(all_num_in_sql)       # 저장된 전체 회차 조회

# # DB에 저장된 최신 회차 정보 조회
# mycursor.execute('SELECT * FROM lotto_nums ORDER BY ID DESC LIMIT 1')
# latest_num_in_sql = mycursor.fetchall()
# print(latest_num_in_sql[0])
# print(f'DB에 저장된 최신회차는 : {latest_num_in_sql} 회차 입니다.')

# ================= 위의 쿼리 진행해보기 =============== #
# 로또 홈페이지 접속
service = Service('../chromedriver')
url = 'https://www.dhlottery.co.kr/gameResult.do?method=byWin'
try:
    driver = webdriver.Chrome(service = service)
except:
    print('chromedriver의 버전을 확인해 주세요 😣')
    time.sleep(2)
    sys.exit()

driver.get(url)


# 최신회차 조회
new = int(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').text.strip().split()[0])
print('최신 회차 : {}회'.format(new))

# 최신회차 검사
if new in all_num_in_sql:
    print('최신 회차 ({})가 DB에 저장되어 있습니다.'.format(new))

else:
    print('최신 회차 ({})가 저장되어 있지 않습니다. (현재 저장되어있는 최신회차 : {}회)'.format(new, all_num_in_sql[0]))
    q = int(input('최신회차 정보를 업데이트 하시겠습니까? (1 : yes, 2: no)'))
    if q == 1:

        for i in range(all_num_in_sql[0]+1, new+1):     # DB에 저장된 최신회차 + 1 부터 신규 회차까지 루프 > 데이터 저장
                        ## 콤보박스에서 맨 처음 숫자 1을 불러오지 못하는 현상 발생 >> 최신부터 역순으로 올라가는 순서를 고려할 것

            driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').send_keys(i)
            driver.implicitly_wait(10)
            driver.find_element(By.XPATH, '//*[@id="searchBtn"]').send_keys(Keys.ENTER)

            nums = []
            for _ in range(1, 7):

                # 신규 회차의 로또번호 6개를 리스트에 넣기
                nums.append(driver.find_element(By.XPATH, '//*[@id="article"]/div[2]/div/div[2]/div/div[1]/p/span[{}]'.format(_)).text)

            # 새로운 회차 및 번호 DB에 저장
            print({i}, '회차가 저장됩니다.', '{}'.format(nums))
            mycursor.execute('INSERT INTO lotto_nums (ID, num1, num2, num3, num4, num5, num6) VALUES ({}, {}, {}, {}, {}, {}, {})'.format(i, nums[0],nums[1],nums[2],nums[3],nums[4],nums[5]))
                                            ## lotto_nums table desc : ID, num1 ~ num6
            mydb.commit()       # 커서의 쿼리 실행 저장

            # DB의 저장된 데이터 확인
            mycursor.execute('SELECT * FROM lotto_nums ORDER BY ID DESC LIMIT 1')
            latest_info = mycursor.fetchall()[0]
            print('{} 회차 : [{}, {}, {}, {}, {}, {}] 저장 됨'.format(latest_info[0], latest_info[1],latest_info[2],latest_info[3],latest_info[4],latest_info[5],latest_info[6]))
            
driver.close()
mydb.close()


최신 회차 : 1024회
최신 회차 (1024)가 DB에 저장되어 있습니다.


In [38]:
## 새로운 테이블 만들기
import mysql
import mysql.connector
from selenium import *
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
import sys, time
import pandas as pd

# ====================== #

# # host 정보 확인
# host = int(input('which is your host? 1. local / 2. server  >> '))
# if host == 1:
#     host = '192.168.35.159'
# elif host == 2:
#     host = input('input server address: ')
# else:
#     sys.exit()
    
mydb = mysql.connector.connect(host = '192.168.35.159', port = '3306', user='', password = '')
mydb

table_name = 'lot_num'      # table name 지정

mycursor = mydb.cursor(buffered = True)


## =========== get db ============= ##
mycursor.execute('USE lotto_db')

# DB 에 새로운 table 만들기
mycursor.execute('CREATE TABLE {} (ID INT NOT NULL AUTO_INCREMENT PRIMARY KEY, NUM1 INT NOT NULL, NUM2 INT NOT NULL, NUM3 INT NOT NULL, NUM4 INT NOT NULL, NUM5 INT NOT NULL, NUM6 INT NOT NULL)'.format(table_name))
mydb.commit()

# TABLE 의 DESC 확인
mycursor.execute('DESC {}'.format(table_name))
mycursor.fetchall()

# DB에 저장된 모든 회차 정보 조회
mycursor.execute('SELECT ID FROM {} ORDER BY ID DESC'.format(table_name))
all_num_in_sql = mycursor.fetchall()
all_num_in_sql = [all_num_in_sql[i][0] for i in range(0,len(all_num_in_sql))]
# print(all_num_in_sql)       # 저장된 전체 회차 조회

# # DB에 저장된 최신 회차 정보 조회
# mycursor.execute('SELECT * FROM {} ORDER BY ID DESC LIMIT 1'.format(table_name))
# latest_num_in_sql = mycursor.fetchall()
# print(latest_num_in_sql[0])
# print(f'TABLE에 저장된 최신회차는 : {latest_num_in_sql} 회차 입니다.')

# ================= 위의 쿼리 진행해보기 =============== #
# 로또 홈페이지 접속
service = Service('../chromedriver')
url = 'https://www.dhlottery.co.kr/gameResult.do?method=byWin'
try:
    driver = webdriver.Chrome(service = service)
except:
    print('chromedriver의 버전을 확인해 주세요 😣')
    time.sleep(2)
    sys.exit()

driver.get(url)


# 최신회차 조회
new = int(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').text.strip().split()[0])
print('최신 회차 : {}회'.format(new))

# 최신회차 검사
if new in all_num_in_sql:
    print('최신 회차 ({})가 DB에 저장되어 있습니다.'.format(new))

elif all_num_in_sql == []:
    print('TABLE 에 로또 정보가 없습니다.')
    q = int(input('웹에서 정보를 받아오시겠습니까? (1 : yes, 2: no) '))
    if q == 1:

        for i in range(1, new+1):     # 1회차 부터 신규 회차까지 루프 > 데이터 저장

            driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').send_keys(i)
            driver.implicitly_wait(10)
            driver.find_element(By.XPATH, '//*[@id="searchBtn"]').send_keys(Keys.ENTER)

            nums = []
            for _ in range(1, 7):

                # 신규 회차의 로또번호 6개를 리스트에 넣기
                nums.append(driver.find_element(By.XPATH, '//*[@id="article"]/div[2]/div/div[2]/div/div[1]/p/span[{}]'.format(_)).text)

            # 새로운 회차 및 번호 DB에 저장
            print({i}, '회차가 저장됩니다.', '{}'.format(nums))
            mycursor.execute('INSERT INTO {} (ID, NUM1, NUM2, NUM3, NUM4, NUM5, NUM6) VALUES ({}, {}, {}, {}, {}, {}, {})'.format(table_name, i, nums[0],nums[1],nums[2],nums[3],nums[4],nums[5]))
                                            ## {table_name} table desc : ID, num1 ~ num6
            mydb.commit()       # 커서의 쿼리 실행 저장

            # DB의 저장된 데이터 확인
            mycursor.execute('SELECT * FROM {} ORDER BY ID DESC LIMIT 1'.format(table_name))
            latest_info = mycursor.fetchall()[0]
            print('{} 회차 : [{}, {}, {}, {}, {}, {}] 저장 됨'.format(latest_info[0], latest_info[1],latest_info[2],latest_info[3],latest_info[4],latest_info[5],latest_info[6]))
            
else:
    print('최신 회차 ({})가 저장되어 있지 않습니다. (현재 저장되어있는 최신회차 : {}회)'.format(new, all_num_in_sql[0]))
    q = int(input('최신회차 정보를 업데이트 하시겠습니까? (1 : yes, 2: no)'))
    if q == 1:

        for i in range(all_num_in_sql[0]+1, new+1):     # DB에 저장된 최신회차 + 1 부터 신규 회차까지 루프 > 데이터 저장

            driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').send_keys(i)
            driver.implicitly_wait(10)
            driver.find_element(By.XPATH, '//*[@id="searchBtn"]').send_keys(Keys.ENTER)

            nums = []
            for _ in range(1, 7):

                # 신규 회차의 로또번호 6개를 리스트에 넣기
                nums.append(driver.find_element(By.XPATH, '//*[@id="article"]/div[2]/div/div[2]/div/div[1]/p/span[{}]'.format(_)).text)

            # 새로운 회차 및 번호 DB에 저장
            print({i}, '회차가 저장됩니다.', '{}'.format(nums))
            mycursor.execute('INSERT INTO {} (ID, NUM1, NUM2, NUM3, NUM4, NUM5, NUM6) VALUES ({}, {}, {}, {}, {}, {}, {})'.format(table_name, i, nums[0],nums[1],nums[2],nums[3],nums[4],nums[5]))
                                            ## {table_name} table desc : ID, num1 ~ num6
            mydb.commit()       # 커서의 쿼리 실행 저장

            # DB의 저장된 데이터 확인
            mycursor.execute('SELECT * FROM {} ORDER BY ID DESC LIMIT 1'.format(table_name))
            latest_info = mycursor.fetchall()[0]
            print('{} 회차 : [{}, {}, {}, {}, {}, {}] 저장 됨'.format(latest_info[0], latest_info[1],latest_info[2],latest_info[3],latest_info[4],latest_info[5],latest_info[6]))
            
driver.close()
mydb.close()


최신 회차 : 1024회
TABLE 에 로또 정보가 없습니다.
{1} 회차가 저장됩니다. ['10', '14', '16', '18', '29', '35']
1 회차 : [10, 14, 16, 18, 29, 35] 저장 됨
{2} 회차가 저장됩니다. ['1', '3', '20', '25', '36', '45']
2 회차 : [1, 3, 20, 25, 36, 45] 저장 됨
{3} 회차가 저장됩니다. ['6', '7', '13', '15', '21', '43']
3 회차 : [6, 7, 13, 15, 21, 43] 저장 됨
{4} 회차가 저장됩니다. ['14', '27', '30', '31', '40', '42']
4 회차 : [14, 27, 30, 31, 40, 42] 저장 됨
{5} 회차가 저장됩니다. ['5', '12', '17', '29', '34', '35']
5 회차 : [5, 12, 17, 29, 34, 35] 저장 됨
{6} 회차가 저장됩니다. ['5', '8', '14', '15', '19', '39']
6 회차 : [5, 8, 14, 15, 19, 39] 저장 됨
{7} 회차가 저장됩니다. ['2', '9', '16', '25', '26', '40']
7 회차 : [2, 9, 16, 25, 26, 40] 저장 됨
{8} 회차가 저장됩니다. ['8', '19', '20', '21', '33', '39']
8 회차 : [8, 19, 20, 21, 33, 39] 저장 됨
{9} 회차가 저장됩니다. ['1', '3', '10', '27', '29', '37']
9 회차 : [1, 3, 10, 27, 29, 37] 저장 됨
{10} 회차가 저장됩니다. ['9', '25', '30', '33', '41', '44']
10 회차 : [9, 25, 30, 33, 41, 44] 저장 됨
{11} 회차가 저장됩니다. ['9', '18', '20', '22', '38', '44']
11 회차 : [9, 18, 20, 22, 38, 44] 저장 됨
{12} 회차가 저

[('lotto_nums',)]

In [71]:
service = Service('../chromedriver')
url = 'https://www.dhlottery.co.kr/gameResult.do?method=byWin'
try:
    driver = webdriver.Chrome(service = service)
except:
    print('chromedriver의 버전을 확인해 주세요 😣')
    time.sleep(2)
    sys.exit()

driver.get(url)


# 최신회차 조회
new = int(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').text.strip().split()[0])
print('최신 회차 : {}회'.format(new))


for i in range(new, 0, -1):     # 신규회차부터 1회차까지 역순으로 데이터 저장

            driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').send_keys(i)
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="searchBtn"]').send_keys(Keys.ENTER)
            time.sleep(1)

            nums = []
            for _ in range(1, 7):

                # 신규 회차의 로또번호 6개를 리스트에 넣기
                nums.append(driver.find_element(By.XPATH, '//*[@id="article"]/div[2]/div/div[2]/div/div[1]/p/span[{}]'.format(_)).text)

            # 새로운 회차 및 번호 DB에 저장
            print({i}, '회차가 저장됩니다.', '{}'.format(nums))
            driver.implicitly_wait(10)
# driver.close()

최신 회차 : 1024회
{1024} 회차가 저장됩니다. ['9', '18', '20', '22', '38', '44']
{1023} 회차가 저장됩니다. ['10', '14', '16', '18', '29', '35']
{1022} 회차가 저장됩니다. ['5', '6', '11', '29', '42', '45']
{1021} 회차가 저장됩니다. ['12', '15', '17', '24', '29', '45']
{1020} 회차가 저장됩니다. ['12', '27', '29', '38', '41', '45']
{1019} 회차가 저장됩니다. ['1', '4', '13', '17', '34', '39']
{1018} 회차가 저장됩니다. ['3', '19', '21', '25', '37', '45']
{1017} 회차가 저장됩니다. ['12', '18', '22', '23', '30', '34']
{1016} 회차가 저장됩니다. ['15', '26', '28', '34', '41', '42']
{1015} 회차가 저장됩니다. ['14', '23', '31', '33', '37', '40']
{1014} 회차가 저장됩니다. ['3', '11', '14', '18', '26', '27']
{1013} 회차가 저장됩니다. ['21', '22', '26', '34', '36', '41']
{1012} 회차가 저장됩니다. ['5', '11', '18', '20', '35', '45']
{1011} 회차가 저장됩니다. ['1', '9', '12', '26', '35', '38']
{1010} 회차가 저장됩니다. ['9', '12', '15', '25', '34', '36']
{1009} 회차가 저장됩니다. ['15', '23', '29', '34', '40', '44']
{1008} 회차가 저장됩니다. ['9', '11', '30', '31', '41', '44']
{1007} 회차가 저장됩니다. ['8', '11', '16', '19', '21', '25']
{1006} 회차

In [ ]:
## 콤보박스의 회차 수가 text로 잡혀있어서 send_key(회차번호) 를 쓰면, 제대로 작동하지 않는 것을 확인
# ex. ~ 1000회 까지는 무난히 가능했지만,
# 999 를 send_key 하는 순간 999의 숫자가 선택되는것이 아니라 997이 선택된다.

# 추론 : 999 라는 번호는 텍스트로 이뤄져 있다. 이에 1000 다음 숫자로 9를 send 하면 999가 선택된다.
# 하지만 그 다음으로 99를 send 하면 997이 선택된다.

## 해결 :   from selenium.webdriver.support.ui import Select 
# select = Select(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]'))
# select.select_by_value(i)
## > value값을 선택하면 된다!

In [165]:
driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').click()
driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').send_keys()

In [109]:
driver.find_element(By.XPATH, '//*[@id="searchBtn"]').send_keys(Keys.ENTER)

In [55]:
nums = []
for _ in range(1, 7):

    # 신규 회차의 로또번호 6개를 리스트에 넣기
    nums.append(driver.find_element(By.XPATH, '//*[@id="article"]/div[2]/div/div[2]/div/div[1]/p/span[{}]'.format(_)).text)

# 새로운 회차 및 번호 DB에 저장
print({i}, '회차가 저장됩니다.', '{}'.format(nums))

{4} 회차가 저장됩니다. ['10', '14', '16', '18', '29', '35']


In [70]:
driver.close()

In [61]:
nums

['6', '30', '38', '39', '40', '43']

In [69]:
service = Service('../chromedriver')
url = 'https://www.dhlottery.co.kr/gameResult.do?method=byWin'
try:
    driver = webdriver.Chrome(service = service)
except:
    print('chromedriver의 버전을 확인해 주세요 😣')
    time.sleep(2)
    sys.exit()

driver.get(url)

In [169]:
from selenium.webdriver.support.ui import Select

select = Select(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]'))

# select.select_by_index(index=2)
# select.select_by_visible_text(text="option_text")
select.select_by_value(value='1')


In [ ]:
from selenium.webdriver.support.ui import Select

service = Service('../chromedriver')
url = 'https://www.dhlottery.co.kr/gameResult.do?method=byWin'
try:
    driver = webdriver.Chrome(service = service)
except:
    print('chromedriver의 버전을 확인해 주세요 😣')
    time.sleep(2)
    sys.exit()

driver.get(url)


# 최신회차 조회
new = int(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').text.strip().split()[0])
print('최신 회차 : {}회'.format(new))


for i in range(new, 0, -1):     # 신규회차부터 1회차까지 역순으로 데이터 저장

            # select = Select(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]'))
            # select.select_by_value(value=i)
            
            driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').send_keys(i)
            time.sleep(1)
            driver.find_element(By.XPATH, '//*[@id="searchBtn"]').send_keys('1000', Keys.ENTER)
            time.sleep(1)

            nums = []
            for _ in range(1, 7):

                # 신규 회차의 로또번호 6개를 리스트에 넣기
                nums.append(driver.find_element(By.XPATH, '//*[@id="article"]/div[2]/div/div[2]/div/div[1]/p/span[{}]'.format(_)).text)

            # 새로운 회차 및 번호 DB에 저장
            print({i}, '회차가 저장됩니다.', '{}'.format(nums))
            driver.implicitly_wait(10)
# driver.close()

In [173]:
# driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').send_keys(i)

driver.find_element(By.XPATH, '//*[@id="searchBtn"]').send_keys('2', Keys.ENTER)


In [185]:
select = Select(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]'))
select.select_by_value(value='999')
driver.find_element(By.XPATH, '//*[@id="searchBtn"]').send_keys(Keys.ENTER)

In [181]:
driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').send_keys('999', Keys.ENTER)

----------------

# 새로운 테이블 만들기

In [212]:
import mysql
import mysql.connector
from selenium import *
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import Select
import sys, time

# ====================== #

# # host 정보 확인
# host = int(input('which is your host? 1. local / 2. server  >> '))
# if host == 1:
#     host = '192.168.35.159'
# elif host == 2:
#     host = input('input server address: ')
# else:
#     sys.exit()
    
mydb = mysql.connector.connect(host = '192.168.35.159', port = '3306', user='', password = '')
mydb

table_name = 'lot_num'      # table name 지정

mycursor = mydb.cursor(buffered = True)


## =========== USE DB ============= ##
mycursor.execute('USE lotto_db')

## =========== Table ======== ##

# DB 에 새로운 TABLE 만들기
mycursor.execute('SHOW TABLES')
find_table = mycursor.fetchall()

if ('{}'.format(table_name),) in find_table:     # TABLE 중복 여부 확인
    print("{} 이라는 테이블이 이미 DB에 존재합니다.".format(table_name))

else:       # TABLE 이 없을 때, 새로 만들기
    mycursor.execute('CREATE TABLE {} (ID INT NOT NULL AUTO_INCREMENT PRIMARY KEY, NUM1 INT NOT NULL, NUM2 INT NOT NULL, NUM3 INT NOT NULL, NUM4 INT NOT NULL, NUM5 INT NOT NULL, NUM6 INT NOT NULL)'.format(table_name))
    mydb.commit()
    print('{} 이라는 테이블을 새로 만들었습니다.'.format(table_name))

# TABLE 의 DESC 확인
mycursor.execute('DESC {}'.format(table_name))
print(mycursor.fetchall())

# DB에 저장된 모든 회차 정보 조회
mycursor.execute('SELECT ID FROM {} ORDER BY ID DESC'.format(table_name))
all_num_in_sql = mycursor.fetchall()
all_num_in_sql = [all_num_in_sql[i][0] for i in range(0,len(all_num_in_sql))]
# print(all_num_in_sql)       # 저장된 전체 회차 조회

# # DB에 저장된 최신 회차 정보 조회
# mycursor.execute('SELECT * FROM {} ORDER BY ID DESC LIMIT 1'.format(table_name))
# latest_num_in_sql = mycursor.fetchall()
# print(latest_num_in_sql[0])
# print(f'TABLE에 저장된 최신회차는 : {latest_num_in_sql} 회차 입니다.')

## ========= Page access ======= ##

# 로또 홈페이지 접속
service = Service('../chromedriver')
url = 'https://www.dhlottery.co.kr/gameResult.do?method=byWin'
try:
    driver = webdriver.Chrome(service = service)
except:
    print('chromedriver의 버전을 확인해 주세요 😣')
    time.sleep(2)
    sys.exit()

driver.get(url)

# 최신회차 조회
new = int(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').text.strip().split()[0])
print('최신 회차 : {}회'.format(new))

# 최신회차 검사
if new in all_num_in_sql:
    print('최신 회차 ({})가 DB에 저장되어 있습니다.'.format(new))
    mycursor.execute('SELECT * FROM {} ORDER BY ID DESC LIMIT 1'.format(table_name))        # 최신회차 정보 불러오기
    latest_nums = list(mycursor.fetchall()[0])
    print('({}회차 : {})'.format(new, latest_nums))

elif all_num_in_sql == []:
    print('TABLE 에 로또 정보가 없습니다.')
    q = int(input('웹에서 정보를 받아오시겠습니까? (1 : yes, 2: no) '))
    if q == 1:

        for i in range(1, new+1):     # 1회차 부터 신규 회차까지 루프 > 데이터 저장

            select = Select(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]'))      # 드롭박스 선택
            select.select_by_value(str(i))       # value 선택

            # driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').send_keys(i)        ### >> 삭제 // 해당 element의 값이 str 형식으로, i키를 받으면 오류가 발생
            driver.implicitly_wait(10)
            driver.find_element(By.XPATH, '//*[@id="searchBtn"]').send_keys(Keys.ENTER)     # 조회버튼 클릭

            nums = []
            for _ in range(1, 7):

                # 신규 회차의 로또번호 6개를 리스트에 넣기
                nums.append(driver.find_element(By.XPATH, '//*[@id="article"]/div[2]/div/div[2]/div/div[1]/p/span[{}]'.format(_)).text)

            # 새로운 회차 및 번호 DB에 저장
            print({i}, '회차가 저장됩니다.', '{}'.format(nums))
            mycursor.execute('INSERT INTO {} (ID, NUM1, NUM2, NUM3, NUM4, NUM5, NUM6) VALUES ({}, {}, {}, {}, {}, {}, {})'.format(table_name, i, nums[0],nums[1],nums[2],nums[3],nums[4],nums[5]))
                                            ## {table_name} table desc : ID, num1 ~ num6
            mydb.commit()       # 커서의 쿼리 실행 저장

            # DB의 저장된 데이터 확인
            mycursor.execute('SELECT * FROM {} ORDER BY ID DESC LIMIT 1'.format(table_name))
            latest_info = mycursor.fetchall()[0]
            print('{} 회차 : [{}, {}, {}, {}, {}, {}] 저장 됨'.format(latest_info[0], latest_info[1],latest_info[2],latest_info[3],latest_info[4],latest_info[5],latest_info[6]))
            
else:
    print('최신 회차 ({})가 저장되어 있지 않습니다. (현재 저장되어있는 최신회차 : {}회)'.format(new, all_num_in_sql[0]))
    q = int(input('최신회차 정보를 업데이트 하시겠습니까? (1 : yes, 2: no)'))
    if q == 1:

        for i in range(all_num_in_sql[0]+1, new+1):     # DB에 저장된 최신회차 + 1 부터 신규 회차까지 루프 > 데이터 저장
            
            select = Select(driver.find_element(By.XPATH, '//*[@id="dwrNoList"]'))      # 드롭박스 선택
            select.select_by_value(str(i))       # value 선택   

            # driver.find_element(By.XPATH, '//*[@id="dwrNoList"]').send_keys(i)        ### >> 삭제 // 해당 element의 값이 str 형식으로, i키를 받으면 오류가 발생
            driver.implicitly_wait(10)
            driver.find_element(By.XPATH, '//*[@id="searchBtn"]').send_keys(Keys.ENTER)     # 조회버튼 클릭

            nums = []
            for _ in range(1, 7):

                # 신규 회차의 로또번호 6개를 리스트에 넣기
                nums.append(driver.find_element(By.XPATH, '//*[@id="article"]/div[2]/div/div[2]/div/div[1]/p/span[{}]'.format(_)).text)

            # 새로운 회차 및 번호 DB에 저장
            print({i}, '회차가 저장됩니다.', '{}'.format(nums))
            mycursor.execute('INSERT INTO {} (ID, NUM1, NUM2, NUM3, NUM4, NUM5, NUM6) VALUES ({}, {}, {}, {}, {}, {}, {})'.format(table_name, i, nums[0],nums[1],nums[2],nums[3],nums[4],nums[5]))
                                            ## {table_name} table desc : ID, num1 ~ num6
            mydb.commit()       # 커서의 쿼리 실행 저장

            # DB의 저장된 데이터 확인
            mycursor.execute('SELECT * FROM {} ORDER BY ID DESC LIMIT 1'.format(table_name))
            latest_info = mycursor.fetchall()[0]
            print('{} 회차 : [{}, {}, {}, {}, {}, {}] 저장 됨'.format(latest_info[0], latest_info[1],latest_info[2],latest_info[3],latest_info[4],latest_info[5],latest_info[6]))
            
driver.close()
mydb.close()


lot_num 이라는 테이블이 이미 DB에 존재합니다.
[('ID', 'int(11)', 'NO', 'PRI', None, 'auto_increment'), ('NUM1', 'int(11)', 'NO', '', None, ''), ('NUM2', 'int(11)', 'NO', '', None, ''), ('NUM3', 'int(11)', 'NO', '', None, ''), ('NUM4', 'int(11)', 'NO', '', None, ''), ('NUM5', 'int(11)', 'NO', '', None, ''), ('NUM6', 'int(11)', 'NO', '', None, '')]
최신 회차 : 1024회
TABLE 에 로또 정보가 없습니다.
{1} 회차가 저장됩니다. ['10', '23', '29', '33', '37', '40']
1 회차 : [10, 23, 29, 33, 37, 40] 저장 됨
{2} 회차가 저장됩니다. ['9', '13', '21', '25', '32', '42']
2 회차 : [9, 13, 21, 25, 32, 42] 저장 됨
{3} 회차가 저장됩니다. ['11', '16', '19', '21', '27', '31']
3 회차 : [11, 16, 19, 21, 27, 31] 저장 됨
{4} 회차가 저장됩니다. ['14', '27', '30', '31', '40', '42']
4 회차 : [14, 27, 30, 31, 40, 42] 저장 됨
{5} 회차가 저장됩니다. ['16', '24', '29', '40', '41', '42']
5 회차 : [16, 24, 29, 40, 41, 42] 저장 됨
{6} 회차가 저장됩니다. ['14', '15', '26', '27', '40', '42']
6 회차 : [14, 15, 26, 27, 40, 42] 저장 됨
{7} 회차가 저장됩니다. ['2', '9', '16', '25', '26', '40']
7 회차 : [2, 9, 16, 25, 26, 40] 저장 됨
{8} 회차가 저장됩니다.

In [214]:
mydb = mysql.connector.connect(host = '192.168.35.159', port = '3306', user='', password = '')
mydb

table_name = 'lot_num'      # table name 지정

mycursor = mydb.cursor(buffered = True)


mycursor.execute('use lotto_db')

mycursor.execute('show tables')
mycursor.fetchall()

mycursor.execute('desc lot_num')
mycursor.fetchall()
# mycursor.execute('drop table lot_num')


[('ID', 'int(11)', 'NO', 'PRI', None, 'auto_increment'),
 ('NUM1', 'int(11)', 'NO', '', None, ''),
 ('NUM2', 'int(11)', 'NO', '', None, ''),
 ('NUM3', 'int(11)', 'NO', '', None, ''),
 ('NUM4', 'int(11)', 'NO', '', None, ''),
 ('NUM5', 'int(11)', 'NO', '', None, ''),
 ('NUM6', 'int(11)', 'NO', '', None, '')]

In [193]:
mydb.close()

In [194]:
driver.close()

In [218]:
mycursor.execute('SELECT * FROM {} ORDER BY ID DESC LIMIT 1'.format(table_name))
list(mycursor.fetchall()[0][1:])

[9, 18, 20, 22, 38, 44]